# Using Fully Connected  NN to Make Predictions

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Preprocessing Plan:
* Use Functional API
* Treat the categorical and numerical features differently.
    * One-hot: Is_Weekday, Is_Weekend, is_ramazan, ramazan_in_7_days, is_kurban, kurban_in_7_days
    * Embed  : Day_Index, curr_month_1_delta, curr_month_15_delta, next_month_1_delta
    * Numerical (normalized): Rest
    * Sources for Functional API:
        * [How to slice an input in keras?](https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras)
        * [Sequential API vs Functional API model in Keras](https://hanifi.medium.com/sequential-api-vs-functional-api-model-in-keras-266823d7cd5e)
    

### Is_Weekday, Is_Weekend, is_ramazan, ramazan_in_7_days, is_kurban, kurban_in_7_days
No change, use directly as one-hot

In [2]:
one_hots = keras.layers.Input(shape=[6])

### Day_Index, curr_month_1_delta, curr_month_15_delta, next_month_1_delta
* [Embedding](https://keras.io/api/layers/core_layers/embedding/): Embed the sparse index vector to dense index

In [3]:
features_to_embed    = keras.layers.Input(shape=[4])
embeddeds            = keras.layers.Embedding(input_dim=4, output_dim=20)(features_to_embed)
flattened_embeddeds  = keras.layers.Flatten(input_shape=[4, 20])(embeddeds)

### Numericals
* [BatchNormalization layer](https://keras.io/api/layers/normalization_layers/batch_normalization/): Normalize each feature. Features are last values, trend and averages of CashIn and CashOut.


In [4]:
features_to_normalize = keras.layers.Input(shape=[60])
normalized            = keras.layers.BatchNormalization()(features_to_normalize)

### Concatenating All Paths Together

In [6]:
preprocessed = keras.layers.Concatenate()([one_hots, flattened_embeddeds, normalized])

Draw the model so far:

In [7]:
preprocessing_model = keras.Model(inputs=[one_hots, features_to_embed, features_to_normalize], outputs=[preprocessed] )
#from tensorflow.keras.utils import plot_model
#plot_model(preprocessing_model, show_shapes=True, show_layer_names=True)
preprocessing_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 4, 20)        80          input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 6)]          0                                            
_______________________________________________________________________________________

## Rest of The Model

## Arranging Data